In [1]:
import networkx as nx
import pandas as pd
from tqdm import tqdm
import os

In [2]:
TRAIN_PATH = '/home/malchul/work/recsys/hackaton/data/train.csv'
TEST_PATH = '/home/malchul/work/recsys/hackaton/data/test.csv'

In [3]:
train_df = pd.read_csv(TRAIN_PATH)

In [ ]:
test_df = pd.read_csv(TEST_PATH)

In [21]:
df = train_df
unique_ego_ids = sorted(df.ego_id.unique())
out_folder = "factorization_train_2"
os.makedirs(out_folder, exist_ok=True)

In [ ]:
import timeit
import time
from scipy.sparse import csr_matrix, coo_array
import implicit
import numpy as np
import logging
import pickle
logging.basicConfig(level=logging.INFO)
for ego_id in tqdm(unique_ego_ids):
    ego_graph_df = df[df.ego_id == ego_id]
    ego_graph_df = ego_graph_df[~pd.isna(ego_graph_df)]
    shape_axis = max(ego_graph_df.u.max(), ego_graph_df.v.max()) + 1
    base_ego_graph_df = ego_graph_df
    ego_graph_df = ego_graph_df.sample(frac=0.8) 
    # Giving rows and columns values
    rows = ego_graph_df.u.to_numpy()
    columns = ego_graph_df.v.to_numpy()
    arrayData = ego_graph_df.x1.to_numpy()
    
    sparse_frienship = csr_matrix((arrayData, (rows, columns)), shape = (shape_axis, shape_axis))

    # initialize a model
    factors = 6
    model = implicit.als.AlternatingLeastSquares(factors=factors, regularization=0.1, alpha=2, iterations=15)
    
    
    # train the model on a sparse matrix of user/item/confidence weights
    model.fit(sparse_frienship, show_progress=False)
    model = model.to_cpu()
    u_vector = model.user_factors
    v_vector = model.item_factors

    feats = np.concatenate([u_vector, v_vector], axis=1)
    assert feats.shape[1] == 2 * factors
    
    score = u_vector.dot(v_vector.T)
    
    # rows = base_ego_graph_df.u.to_numpy()
    # columns = base_ego_graph_df.v.to_numpy()
    # arrayData = base_ego_graph_df.x1.to_numpy()
    # sparse_frienship = csr_matrix((arrayData, (rows, columns)), shape = (shape_axis, shape_axis))
    
    # print((u_vector.dot(v_vector.T)).max(), sparse_frienship.max())
    # print(np.mean(np.abs((u_vector.dot(v_vector.T) * (sparse_frienship > 0) - sparse_frienship))[(sparse_frienship > 0).todense()]), np.sum((sparse_frienship > 0)), np.sum(sparse_frienship == 0))
    # print(np.abs((u_vector.dot(v_vector.T))[(sparse_frienship > 0).todense()]))
    # print(np.abs(sparse_frienship)[(sparse_frienship > 0).todense()])
    # break
    import pickle
    data = {
        'scores': score,
        'u': u_vector,
        'v': v_vector
    }
    
    with open(os.path.join(out_folder, f'{ego_id}.pickle'), 'wb') as f:
        pickle.dump(data, f)
    

 20%|████████████████████████████████████▋                                                                                                                                                  | 12368/61786 [25:08<3:12:10,  4.29it/s]